# Advanced Machine Learning - Project 1

## TODO
- renaming vars
- nan imputation 
- check how to pre-process features (How to encode categorical data?, How to scale continuous?)
- read into hybrid Bayesian Networks? How to do?
- train test split, do KFold?

## Learning approach
- (maybe feature selection)
1. learn structure
2. learn CPDs

## Make predictions

## Interpret

In [1]:
# Imports

# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

# Data Upscaling or Downscaling
from sklearn.model_selection import train_test_split
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling

# Bayesian network and estimators
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

from pgmpy.estimators import ParameterEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import BayesianEstimator

from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch




d:\OneDrive - Universidade de Coimbra\Mestrado\Files\1Semestre\AAA\P\P1\venv_AML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reading the dataset
df = pd.read_csv("../CHD_dataset.csv")

df.head(10)

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,male,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,female,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,male,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,female,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,female,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
5,female,43,2.0,No,0.0,0.0,0,1,No,228.0,180.0,110.0,30.30,77.0,99.0,0
6,female,63,1.0,No,0.0,0.0,0,0,No,205.0,138.0,71.0,33.11,60.0,85.0,1
7,female,45,2.0,Yes,20.0,0.0,0,0,No,313.0,100.0,71.0,21.68,79.0,78.0,0
8,male,52,1.0,No,0.0,0.0,0,1,No,260.0,141.5,89.0,26.36,76.0,79.0,0
9,male,43,1.0,Yes,30.0,0.0,0,1,No,225.0,162.0,107.0,23.61,93.0,88.0,0


In [30]:
# Check some stats for continuous variables cols

for i in df.loc[: ,("age","cigsPerDay","BPMeds","totChol","sysBP","diaBP","BMI","heartRate","glucose")]:
    print(f'{i}: min: {df[i].min()}, max: {df[i].max()}, mean: {df[i].mean()}, std: {df[i].std()}')

age: min: 32, max: 70, mean: 49.58018867924528, std: 8.57294217547334
cigsPerDay: min: 0.0, max: 70.0, mean: 9.069889404372683, std: 11.906581103907081
BPMeds: min: 0.0, max: 1.0, mean: 0.029648743869584437, std: 0.1684837434559677
totChol: min: 107.0, max: 696.0, mean: 236.7213938753834, std: 44.32861687004747
sysBP: min: 83.5, max: 295.0, mean: 132.35459905660377, std: 22.033299608849184
diaBP: min: 48.0, max: 142.5, mean: 82.89775943396226, std: 11.91039448330595
BMI: min: 15.54, max: 56.8, mean: 25.80029641305541, std: 4.070742118400298
heartRate: min: 44.0, max: 143.0, mean: 75.87859841609013, std: 12.023955275467797
glucose: min: 40.0, max: 394.0, mean: 81.9597973709634, std: 22.831786192895972


In [3]:
# NaN imputation

# Checking for NaN
# print(df.isnull().sum())

# Deleting rows with multiple NaN values (in this case, it's none)
df.dropna(thresh = 2, inplace=True)

#TODO probably we can do a function for reducing this piece of code

# Get the mean of cigs per day of smokers
mean_cigs_per_day = df[ (df["currentSmoker"] == "Yes") & ( df["cigsPerDay"] > 0 ) ]["cigsPerDay"].mean(skipna=True)

# Fill the NaN values of smokers with the mean, of non smokers with 0
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "Yes") , "cigsPerDay" ] =  mean_cigs_per_day
df.loc[ (df["cigsPerDay"].isna()) & ( df["currentSmoker"] == "No"), "cigsPerDay" ] = 0


# Get mean BMI for male and female
mean_bmi_male = df.loc[ df["Sex"] == "male", "BMI"].mean()
mean_bmi_female = df.loc[ df["Sex"] == "female", "BMI"].mean()

# Fill NaN values
df.loc[ ( df["Sex"] == "male" ) & ( df["BMI"].isna() ), "BMI"] = mean_bmi_male
df.loc[ ( df["Sex"] == "female" ) & ( df["BMI"].isna() ), "BMI"] = mean_bmi_female


# Get mean BPMeds
mean_bpmeds_male = df.loc[ (df["BPMeds"].notna()) & ( df["Sex"] == "male"), "BPMeds"].mean()
mean_bpmeds_female = df.loc[ (df["BPMeds"].notna()) & (df["Sex"] == "female"), "BPMeds"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["BPMeds"].isna() ), "BPMeds"] = mean_bpmeds_male
df.loc[ (df["Sex"] == "female") & ( df["BPMeds"].isna() ), "BPMeds"] = mean_bpmeds_female


# Get mean totChol
mean_totChol_male = df.loc[ (df["totChol"].notna()) & (df["Sex"] == "male"), "totChol"].mean()
mean_totChol_female = df.loc[ (df["totChol"].notna()) & (df["Sex"] == "female"), "totChol"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["totChol"].isna() ), "totChol"] = mean_totChol_male
df.loc[ (df["Sex"] == "female") & ( df["totChol"].isna() ), "totChol"] = mean_totChol_female


# Get mean glucose
mean_glucose_male = df.loc[ (df["glucose"].notna()) & (df["Sex"] == "male"), "glucose"].mean()
mean_glucose_female = df.loc[ (df["glucose"].notna()) & (df["Sex"] == "female"), "glucose"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["glucose"].isna() ), "glucose"] = mean_glucose_male
df.loc[ (df["Sex"] == "female") & ( df["glucose"].isna() ), "glucose"] = mean_glucose_female


# Get mean heartRate
mean_heartRate_male = df.loc[ (df["heartRate"].notna()) & (df["Sex"] == "male"), "heartRate"].mean()
mean_heartRate_female = df.loc[ (df["heartRate"].notna()) & (df["Sex"] == "female"), "heartRate"].mean()

# Fill NaN values
df.loc[ (df["Sex"] == "male") & ( df["heartRate"].isna() ), "heartRate"] = mean_heartRate_male
df.loc[ (df["Sex"] == "female") & ( df["heartRate"].isna() ), "heartRate"] = mean_heartRate_female


# Education -> should we do clustering??  TODO: im not sure about using mean in this variable!!
mean_education = df.loc[ df["education"].notna(), "education"].mean()

df.loc[ df["education"].isna(), "education"] = mean_education

# Checking if NaN are corrected 
# print(df.isnull().sum())

In [33]:
# Convert categorical data to numerical
# Education, Age, cigsPerDay, BPMeds, totChol, sysBP, diaBP, BMI, heartRate, glucose

def binarize( col, labels, number_bins: int):
    bins = np.linspace( col.min(), col.max(), number_bins+1 )
    # print(f'{col.min()} {col.max()}')
    # print(bins)
    return pd.cut( col, bins, labels=labels, include_lowest=True )


# Education -> converting to integers (categorical) ?!??!?!?! not working for some reason, none of the methods under
#df.loc[ : , "education" ] = df["education"].astype('int32')
# df.loc[:,"education"] = df["education"].apply(lambda x: int(x))
# print(df["education"])


# Get the bins for each continuous variable
bins_education = binarize (df["education"], ["Some High School", "High School/GED", "Some College/Vocational School", "College"], 4)

bins_cigsPerDay = binarize (df["cigsPerDay"], ["Almost Non Smoker","Mid Smoker", "Smoker"], 3) # need to create a class just for non smokers

bins_BPMeds = binarize (df["BPMeds"], ["No", "Yes"], 2) # This was based on min, max and std values

bins_totChol = binarize (df["totChol"], ["Low", "Low-Normal", "Normal-High", "High"], 4)

bins_sysBP = binarize (df["sysBP"], ["Low", "Normal", "High"], 3)

bins_diaBP = binarize (df["diaBP"], ["Low", "Normal", "High"], 3)

bins_BMI = binarize (df["BMI"], ["Underweight", "Normal", "Overweight", "Obese"], 4) # This variable should be divided in other way?

bins_heartRate = binarize (df["heartRate"], ["Low", "Normal", "High"], 3)

bins_glucose = binarize (df["glucose"], ["Low", "Normal", "High"], 3)

# Add the new columns to the dataframe and remove the continuous ones
test_df = df.drop(columns=["education", "cigsPerDay", "BPMeds", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]).assign(cat_education=bins_education, cat_cigsPerDay=bins_cigsPerDay, cat_BPMeds=bins_BPMeds, cat_totChol=bins_totChol, cat_sysBP=bins_sysBP, cat_diaBP=bins_diaBP, cat_BMI=bins_BMI, cat_heartRate=bins_heartRate, cat_glucose=bins_glucose)

print("Shape "+ str(test_df.shape))
print(test_df.columns)
print(test_df.head(10))


Shape (4240, 16)
Index(['Sex', 'age', 'currentSmoker', 'prevalentStroke', 'prevalentHyp',
       'diabetes', 'TenYearCHD', 'cat_education', 'cat_cigsPerDay',
       'cat_BPMeds', 'cat_totChol', 'cat_sysBP', 'cat_diaBP', 'cat_BMI',
       'cat_heartRate', 'cat_glucose'],
      dtype='object')
      Sex  age currentSmoker  prevalentStroke  prevalentHyp diabetes  \
0    male   39            No                0             0       No   
1  female   46            No                0             0       No   
2    male   48           Yes                0             0       No   
3  female   61           Yes                0             1       No   
4  female   46           Yes                0             0       No   
5  female   43            No                0             1       No   
6  female   63            No                0             0       No   
7  female   45           Yes                0             0       No   
8    male   52            No                0             1

In [ ]:
# Scaling (Do we need scaling if every variable is categorical?!??!?!?)

In [39]:
# TESTING STRUCTURE LEARNING

bdeu = BDeuScore(test_df[:2], equivalent_sample_size=5)

es = ExhaustiveSearch(test_df[:2], scoring_method=bdeu)
best_model = es.estimate()
print(best_model.edges())


      Sex  age currentSmoker  prevalentStroke  prevalentHyp diabetes  \
0    male   39            No                0             0       No   
1  female   46            No                0             0       No   
2    male   48           Yes                0             0       No   

   TenYearCHD     cat_education     cat_cigsPerDay cat_BPMeds cat_totChol  \
0           0           College  Almost Non Smoker         No         Low   
1           0   High School/GED  Almost Non Smoker         No         Low   
2           0  Some High School  Almost Non Smoker         No         Low   

  cat_sysBP cat_diaBP      cat_BMI cat_heartRate cat_glucose  
0       Low       Low       Normal        Normal         Low  
1       Low    Normal       Normal        Normal         Low  
2       Low    Normal  Underweight           Low         Low  


In [ ]:
# Remove target variable from the dataset
# Split into train and test data (scikit)